## Building Recommendation engines using pyspark 

### Recommendations Are Everywhere!!

# Dataset

20 million ratings and 465,000 tag applications applied to 27,000 movies by 138,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags. Released 4/2015; updated 10/2016 to update links.csv and add tag genome data.

source : https://grouplens.org/datasets/movielens/

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('RS') \
    .getOrCreate()

# Reading the data

In [2]:
ratings = spark.read\
            .format('csv')\
            .option('header', 'true')\
            .load('ml-20m/ratings.csv')

In [3]:
ratings.columns

['userId', 'movieId', 'rating', 'timestamp']

In [4]:
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
+------+-------+------+----------+
only showing top 5 rows



# EDA

### Calculate sparsity
As you know, ALS works well with sparse datasets. Let's see how much of the ratings matrix is actually empty.
<img src="sparsity.png" >

In [5]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
#The 1.0 is added to ensure the sparsity is returned as a decimal and not an integer.

sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.46% empty.


In [6]:
# Group data by userId, count ratings
ratings.groupBy("userId").count().show()

+------+-----+
|userId|count|
+------+-----+
|   296|   25|
|   467|   30|
|   675|  187|
|   691|   35|
|   829|  387|
|  1090|   74|
|  1159|  235|
|  1436|  234|
|  1512|   68|
|  1572|   64|
|  2069|   45|
|  2088|   87|
|  2136|  201|
|  2162|  100|
|  2294|   21|
|  2904|   23|
|  3210|  452|
|  3414|   29|
|  3606|   66|
|  3959|   24|
+------+-----+
only showing top 20 rows



### MovieLens Summary Statistics

In [17]:
# Min num ratings for movies
print("Movie with the fewest ratings: ")
ratings.groupBy("movieId").count().agg({"count": "min"}).show()

Movie with the fewest ratings: 
+----------+
|min(count)|
+----------+
|         1|
+----------+



In [19]:

# Avg num ratings per movie
print("Avg num ratings per movie: ")
ratings.groupBy("movieId").agg({"rating": "avg"}).show()



Avg num ratings per movie: 
+-------+------------------+
|movieId|       avg(rating)|
+-------+------------------+
|    296| 4.174231169217055|
|   1090| 3.919977226720648|
|   3959| 3.699372603694667|
|   2294| 3.303207714257601|
|   6731|3.5571184995737424|
|  48738| 3.895868364160461|
|   3210|3.6711219879518073|
|  88140|3.5536100302637266|
|    467|3.3832658569500675|
|   2088| 2.562729584628426|
|   2069| 3.806294326241135|
|  50802|  2.85519801980198|
|    829|2.6765513454146075|
|   2136| 2.849462365591398|
|  89864|3.8558174523570714|
|   2904|3.5884353741496597|
|   4821|3.1852010265183917|
|  62912|2.3253676470588234|
|  55498|2.9166666666666665|
|   2162|2.4223394055608822|
+-------+------------------+
only showing top 20 rows



In [21]:
# Min num ratings for user
print("User with the fewest ratings: ")
ratings.groupBy("userId").count().agg({"count": "min"}).show()

User with the fewest ratings: 
+----------+
|min(count)|
+----------+
|        20|
+----------+



## Checking Schema
Spark's implementation of ALS requires that movieIds and userIds be provided as numeric datatypes. Many datasets need to be prepared accordingly in order for them to function properly with Spark. A common issue is that Spark thinks numbers are strings, and vice versa.

In [7]:
#We have a problem in the schema
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [8]:
#let's fix it
# Tell Spark to convert the columns to the proper data types
ratings = ratings.select(ratings.userId.cast("integer"), ratings.movieId.cast("integer"), \
                         ratings.rating.cast("double"))

# Call .printSchema() again to confirm the columns are now in the correct format
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



#  The model

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

(training_data, test_data) = ratings.randomSplit([0.8, 0.2], seed=42)

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", rank =10, maxIter =15, regParam =0.1,
          coldStartStrategy="drop", nonnegative =True, implicitPrefs = False)

# Fit the mdoel to the training_data
model = als.fit(training_data)

# Generate predictions on the test_data
test_predictions = model.transform(test_data)
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
| 20132|    148|   3.0| 2.6610093|
| 22884|    148|   3.0| 2.4764576|
| 96427|    148|   3.0|   3.07394|
|  1259|    148|   5.0|  3.346315|
| 60081|    148|   2.0| 2.8290722|
|130122|    148|   3.0|  2.803703|
| 36445|    148|   4.5| 2.3015115|
| 46146|    148|   2.0| 1.8508147|
| 46944|    148|   2.0| 2.8999782|
| 60334|    148|   4.0| 2.9264936|
| 64843|    148|   3.5|  2.607587|
|101628|    148|   1.0| 2.9659145|
|111523|    148|   2.0|  3.249845|
| 23115|    148|   1.0|  2.847929|
| 61815|    148|   3.0| 3.5685768|
|109910|    148|   2.0| 2.2265484|
|  9084|    148|   2.0| 2.8719149|
| 86098|    148|   3.0| 2.7471077|
| 65304|    148|   2.0| 2.9384236|
| 17655|    148|   4.0| 3.1932292|
+------+-------+------+----------+
only showing top 20 rows



#### Build RMSE evaluator

In [14]:
# Complete the evaluator code
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Extract the 3 parameters
print(evaluator.getMetricName())
print(evaluator.getLabelCol())
print(evaluator.getPredictionCol())

rmse
rating
prediction


In [16]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
| 20132|    148|   3.0| 2.6610093|
| 22884|    148|   3.0| 2.4764576|
| 96427|    148|   3.0|   3.07394|
|  1259|    148|   5.0|  3.346315|
| 60081|    148|   2.0| 2.8290722|
|130122|    148|   3.0|  2.803703|
| 36445|    148|   4.5| 2.3015115|
| 46146|    148|   2.0| 1.8508147|
| 46944|    148|   2.0| 2.8999782|
| 60334|    148|   4.0| 2.9264936|
| 64843|    148|   3.5|  2.607587|
|101628|    148|   1.0| 2.9659145|
|111523|    148|   2.0|  3.249845|
| 23115|    148|   1.0|  2.847929|
| 61815|    148|   3.0| 3.5685768|
|109910|    148|   2.0| 2.2265484|
|  9084|    148|   2.0| 2.8719149|
| 86098|    148|   3.0| 2.7471077|
| 65304|    148|   2.0| 2.9384236|
| 17655|    148|   4.0| 3.1932292|
+------+-------+------+----------+
only showing top 20 rows



#### Model performance evaluation

In [15]:
# Evaluate the "test_predictions" dataframe
RMSE = evaluator.evaluate(test_predictions)

# Print the RMSE
print (RMSE)

0.8117289044037578


#### An RMSE of 0.811 means that on average the model predicts 0.811 above or below values of the original ratings matrix.

#### Get Recommendations

recommendForAllUsers(N) : is native spark function that gets top N recommendations for each user

In [18]:
als_recommendations = model.recommendForAllUsers(5)

#### the output is not very human readable

In [19]:
als_recommendations.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   148|[[126219, 5.89320...|
|   463|[[126219, 6.24572...|
|   471|[[126219, 5.58470...|
|   496|[[121029, 6.21010...|
|   833|[[126219, 6.10001...|
|  1088|[[77736, 5.424657...|
|  1238|[[126219, 6.04946...|
|  1342|[[128830, 6.24111...|
|  1580|[[93008, 4.587152...|
|  1591|[[126219, 7.16933...|
|  1645|[[121029, 5.64239...|
|  1829|[[121029, 6.50329...|
|  1959|[[121029, 5.33813...|
|  2122|[[128830, 4.78646...|
|  2142|[[126219, 6.49672...|
|  2366|[[126219, 6.54641...|
|  2659|[[121029, 5.81776...|
|  2866|[[126219, 5.59148...|
|  3175|[[126219, 8.15337...|
|  3749|[[126219, 6.24814...|
+------+--------------------+
only showing top 20 rows



#### Cleaning up recommendation output

In [23]:
als_recommendations.registerTempTable("ALS_recs_temp")

#### now it is readable 

In [26]:
clean_recs = spark.sql("SELECT userId,\
                       movieIds_and_ratings.movieId AS movieId,\
                       movieIds_and_ratings.rating AS prediction\
                       FROM ALS_recs_temp\
                       LATERAL VIEW explode(recommendations) exploded_table\
                       AS movieIds_and_ratings")

In [27]:
clean_recs.show(10)

+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|   148| 126219| 5.8932056|
|   148| 120821| 5.8524375|
|   148| 121029|  5.594061|
|   148|  77736|  5.418714|
|   148|  86237| 5.4100146|
|   463| 126219|  6.245726|
|   463| 117907| 5.7048626|
|   463| 121029| 5.6821766|
|   463|  77736|  5.529721|
|   463| 128830| 5.5152845|
+------+-------+----------+
only showing top 10 rows



#### And now we will join it with the movie info to be more readable 

In [29]:
movie_info = spark.read\
            .format('csv')\
            .option('header', 'true')\
            .load('ml-20m/movies.csv')

In [30]:
movie_info.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



##### now we can see what's going on even better

In [31]:
clean_recs.join(movie_info, ["movieId"],"left").show()

+-------+------+----------+--------------------+------------------+
|movieId|userId|prediction|               title|            genres|
+-------+------+----------+--------------------+------------------+
| 126219|   148| 5.8932056|    Marihuana (1936)| Documentary|Drama|
| 120821|   148| 5.8524375|The War at Home (...|   Documentary|War|
| 121029|   148|  5.594061|No Distance Left ...|       Documentary|
|  77736|   148|  5.418714|Crazy Stone (Feng...|      Comedy|Crime|
|  86237|   148| 5.4100146|  Connections (1978)|       Documentary|
| 126219|   463|  6.245726|    Marihuana (1936)| Documentary|Drama|
| 117907|   463| 5.7048626|My Brother Tom (2...|             Drama|
| 121029|   463| 5.6821766|No Distance Left ...|       Documentary|
|  77736|   463|  5.529721|Crazy Stone (Feng...|      Comedy|Crime|
| 128830|   463| 5.5152845|  Plastic Bag (2009)|             Drama|
| 126219|   471| 5.5847054|    Marihuana (1936)| Documentary|Drama|
|  77736|   471| 5.4270425|Crazy Stone (Feng...|

#### And if we added filter after joinning with rating we can see nulls as the model predicts ratings for the movies that the individual user has not seen

In [35]:
clean_recs.join(ratings, ["userId","movieId"],"left")\
.filter(ratings.rating.isNull()).show()

+------+-------+----------+------+
|userId|movieId|prediction|rating|
+------+-------+----------+------+
|     9| 126219| 5.8424616|  null|
|    17| 126219|  7.378057|  null|
|    19| 121029|  6.148024|  null|
|   192| 121029|  6.768174|  null|
|   353| 117907|  5.402604|  null|
|   370| 129536| 5.2093616|  null|
|   377|  77736|   5.47226|  null|
|   416|  77736| 5.5944695|  null|
|   452| 126219| 6.9562807|  null|
|   464|  77736| 4.2560515|  null|
|   477| 126219| 5.8053718|  null|
|   535| 121029| 5.9514947|  null|
|   628| 129536| 5.6074753|  null|
|   667| 107890|   5.14787|  null|
|   670| 121029|  6.206175|  null|
|   686| 126219|  5.537974|  null|
|   715|  77736|  5.404034|  null|
|   721| 117907|  5.650877|  null|
|   731| 121029|  5.678696|  null|
|   739|  77736|  5.632146|  null|
+------+-------+----------+------+
only showing top 20 rows



### Do Recommendations Make Sense?

In [36]:
original_ratings = ratings.join(movie_info, ["movieId"],"left")
original_ratings.show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|      2|     1|   3.5|      Jumanji (1995)|Adventure|Childre...|
|     29|     1|   3.5|City of Lost Chil...|Adventure|Drama|F...|
|     32|     1|   3.5|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|
|     47|     1|   3.5|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|     1|   3.5|Usual Suspects, T...|Crime|Mystery|Thr...|
|    112|     1|   3.5|Rumble in the Bro...|Action|Adventure|...|
|    151|     1|   4.0|      Rob Roy (1995)|Action|Drama|Roma...|
|    223|     1|   4.0|       Clerks (1994)|              Comedy|
|    253|     1|   4.0|Interview with th...|        Drama|Horror|
|    260|     1|   4.0|Star Wars: Episod...|Action|Adventure|...|
|    293|     1|   4.0|Léon: The Profess...|Action|Crime|Dram...|
|    296|     1|   4.0| Pulp Fiction (1994)|Comedy|Crime|Dram...|
|    318| 

#### Notice the genre!

In [39]:
from pyspark.sql.functions import col

# Look at user 60's ratings
print("User 60's Ratings:")
original_ratings.filter(col("userId") == 60).sort("rating", ascending = False).show()

# Look at the movies recommended to user 60
print("User 60s Recommendations:")
als_recommendations.filter(col("userId") == 60).show()


User 60's Ratings:
+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   5816|    60|   5.0|Harry Potter and ...|   Adventure|Fantasy|
|   1197|    60|   5.0|Princess Bride, T...|Action|Adventure|...|
|  26760|    60|   5.0|Wild Hearts Can't...|       Drama|Romance|
|  33358|    60|   5.0|  Off the Map (2003)|        Comedy|Drama|
|  40815|    60|   5.0|Harry Potter and ...|Adventure|Fantasy...|
|   1967|    60|   5.0|    Labyrinth (1986)|Adventure|Fantasy...|
|  54001|    60|   5.0|Harry Potter and ...|Adventure|Drama|F...|
|  55247|    60|   5.0|Into the Wild (2007)|Action|Adventure|...|
|  69844|    60|   5.0|Harry Potter and ...|Adventure|Fantasy...|
|  71282|    60|   5.0|   Food, Inc. (2008)|         Documentary|
|  81834|    60|   5.0|Harry Potter and ...|Action|Adventure|...|
|  88125|    60|   5.0|Harry Potter and ...|Action|Advent

### Using gridsearch and cross validation

this is another way to build the model with hyperparameter tunning 


note : computationaly expensive

In [25]:
# Import the required functions

# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False)

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [27]:

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
           .addGrid(als.rank, [10, 50, 75, 100]) \
           .addGrid(als.maxIter, [5, 50, 100, 200]) \
           .addGrid(als.regParam, [.01, .05, .1, .15]) \
           .build()

# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  64


#### Build your cross validation pipeline
Now that we have our data, our train/test splits, our model, and our hyperparameter values, let's tell Spark how to cross validate our model so it can find the best combination of hyperparameters and return it to us.

In [28]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_f0b7f744c192


#### Best Model and Best Model Parameters

In [ ]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

In [ ]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# Print "Rank"
print("  Rank:", best_model.getRank())

# Print "MaxIter"
print("  MaxIter:", best_model.getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model.getRegParam())

###### This is the output when the model has been run over powerful machine



**Best Model**
      Rank: 50
      MaxIter: 100
      RegParam: 0.1